필요한 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import tempfile

환경 설정 및 데이터 불러오기기

In [ ]:
# 환경 변수 설정 (Unicode 인코딩 문제 해결)
os.environ["PYTHONIOENCODING"] = "utf-8"

# 임시 디렉토리 설정 (한글 없는 경로로 설정)
temp_dir = tempfile.mkdtemp(dir='C:\\temp')
os.environ['JOBLIB_TEMP_FOLDER'] = temp_dir

# 폰트 설정 - 윈도우 기본 한글 폰트 적용
font_path = 'C:/Windows/Fonts/Malgun.ttf'  
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 부호 깨짐 방지

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

결측치 처리

In [ ]:
# 결측치 처리
train = train.assign(배터리용량=train['배터리용량'].fillna(train['배터리용량'].mean()))
test = test.assign(배터리용량=test['배터리용량'].fillna(train['배터리용량'].mean()))

데이터 전처리

In [ ]:
x_train = train.drop(columns=['가격(백만원)'])
y_train = train['가격(백만원)']
x_test = test.copy()

범주형 데이터 인코딩

In [ ]:
# 범주형 변수 레이블 인코딩
categorical_features = [col for col in x_train.columns if x_train[col].dtype == 'object']

for col in categorical_features:
    le = LabelEncoder()
    x_train[col] = le.fit_transform(x_train[col])
    for case in np.unique(x_test[col]):
        if case not in le.classes_:
            le.classes_ = np.append(le.classes_, case)
    x_test[col] = le.transform(x_test[col])

특성 스케일링

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

모델 학습 및 하이퍼파라미터 튜닝

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# 학습
grid_search.fit(x_train_scaled, y_train)
best_model = grid_search.best_estimator_

모델 평가 (훈련 데이터 RMSE)

In [ ]:
y_pred_train = best_model.predict(x_train_scaled)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"훈련 데이터 RMSE: {rmse_train:.4f}")


테스트 데이터 예측 및 결과 저장

In [ ]:
# 테스트 데이터 예측 수행
pred = best_model.predict(x_test_scaled)

# 결과 저장
submit = pd.read_csv('./sample_submission.csv')
submit['가격(백만원)'] = pred
submit.to_csv('./optimized_submission.csv', index=False)

print("제출 파일이 성공적으로 저장되었습니다.")